In [1]:
!pip -q install transformers datasets accelerate adapter-transformers bitsandbytes
!git clone https://github.com/jonathan-hsu123/LLM-Adapters.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 

In [2]:
%cd LLM-Adapters
!pip install -r requirements.txt

/content/LLM-Adapters
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-cbpgzt8u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-cbpgzt8u
  Resolved https://github.com/huggingface/transformers.git to commit bdb29ff9f3b8030772bd4be037d061f253c0e928
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:

In [3]:
%cd peft
!pip install .

/content/LLM-Adapters/peft
Processing /content/LLM-Adapters/peft
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=40990 sha256=4b119bde01d7b452505132f417fb845922e76d1384fbb9237dcf75e8c33f2d31
  Stored in directory: /tmp/pip-ephem-wheel-cache-zgvyhnn3/wheels/4a/97/82/f1028d6061960ab6b5a8eed03ee4008d1ccc1c427c47b665d4
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.13.2
    Uninstalling peft-0.13.2:
      Successfully uninstalled peft-0.13.2


In [ ]:
from datasets import load_dataset
dataset = load_dataset("boolq")

README.md:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.69M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [ ]:
def format_example(example):
    question = example['question']
    passage = example['passage']
    answer = 'Yes' if example['answer'] else 'No'
    prompt = f"Question: {question}\nPassage: {passage}\nAnswer:"
    return {'input_text': prompt, 'target_text': answer}
formatted_dataset = dataset.map(format_example, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = [f"{input_text} {target_text}" for input_text, target_text in zip(examples['input_text'], examples['target_text'])]

    # Tokenize the combined text and return PyTorch tensors
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt',  # Ensure tensors are returned
    )

    # Create labels by cloning the input IDs
    labels = model_inputs['input_ids'].clone()

    # Determine the length of the input_text for each example
    input_lengths = []
    for input_text in examples['input_text']:
        input_ids = tokenizer(
            input_text,
            max_length=512,
            truncation=True,
            add_special_tokens=False,  # Do not add special tokens here
        )['input_ids']
        input_lengths.append(len(input_ids))

    # Mask the labels for the input_text portion
    for i, input_length in enumerate(input_lengths):
        labels[i, :input_length] = -100  # Use -100 to ignore the input tokens

    model_inputs['labels'] = labels

    return model_inputs

tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset['train'].column_names,
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We are not using masked language modeling
)

In [4]:
# model = LlamaForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf', load_in_8bit=True, device_map='auto')
from transformers import GPT2Tokenizer, GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')
from peft import BottleneckConfig, get_peft_model

config = BottleneckConfig(
    bottleneck_size=256,
    non_linearity="tanh",
    adapter_dropout=0.0,
    use_parallel_adapter=True,
    use_adapterp=False,
    scaling=1.0,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn"],
)
model = get_peft_model(model, config)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)
Conv1D(nf=2304, nx=768)


In [5]:
# Detailed parameter counting
total_params = 0
trainable_params = 0
non_trainable_params = 0

for name, parameter in model.named_parameters():
    param_count = parameter.numel()
    total_params += param_count

    if parameter.requires_grad:
        trainable_params += param_count
    else:
        non_trainable_params += param_count

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {non_trainable_params:,}")
print(f"percentage: {(trainable_params/total_params)*100:.4f}%")

Total parameters: 133,907,712
Trainable parameters: 9,467,904
Non-trainable parameters: 124,439,808
percentage: 7.0705%


In [ ]:
from transformers import TrainingArguments

# Set the pad_token to the eos_token
tokenizer.pad_token = tokenizer.eos_token

# Update model's config
model.config.pad_token_id = tokenizer.pad_token_id
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    # evaluation_strategy='steps',
    # eval_steps=1000,
    save_steps=3000,
    logging_dir='./logs',
    logging_steps=1000,
    learning_rate=1e-4,
    report_to='none',
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-15-d59db983b582>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
1000,3.061600
2000,2.940800
3000,2.906100
4000,2.877300
5000,2.856700
6000,2.831100
7000,2.814600
8000,2.801600
9000,2.786100
10000,2.779100


TrainOutput(global_step=11790, training_loss=2.851407774067006, metrics={'train_runtime': 11337.2741, 'train_samples_per_second': 8.315, 'train_steps_per_second': 1.04, 'total_flos': 2.737388061720576e+16, 'train_loss': 2.851407774067006, 'epoch': 10.0})

In [ ]:
from datasets import load_dataset

# Load the validation split
dataset = load_dataset("boolq", split='validation')
formatted_dataset = dataset.map(format_example)

In [ ]:
import torch
model.eval()

def generate_answer(example):
    input_text = example['input_text']

    # Tokenize and move inputs to the model's device
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        max_length=512,
        truncation=True,
        padding='max_length',
    )
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    # Generate the model's output
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=15,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode the generated tokens
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract the answer by removing the input prompt
    answer = generated_text[len(input_text):].strip()

    # Handle empty outputs
    if not answer:
        answer = "No"
        return {'predicted_answer': answer}

    # # Keep only the first word (Yes or No)
    # answer = answer.split()[0]
    # print(answer)

    # # Handle unexpected outputs
    if 'yes' in answer.lower():
        answer = 'Yes'
    elif 'no' in answer.lower():
        answer = 'No'
    else:
        answer = 'No'

    return {'predicted_answer': answer}


# Map the generate_answer function over the validation set
predictions = formatted_dataset.map(generate_answer, batched=False)


Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [ ]:
import numpy as np

def compute_accuracy(predictions):
    # Convert the true and predicted answers to lowercase for consistency
    true_answers = [answer.lower() for answer in predictions['target_text']]
    predicted_answers = [answer.lower() for answer in predictions['predicted_answer']]

    # Calculate the number of correct predictions
    correct_predictions = sum([
        true == pred for true, pred in zip(true_answers, predicted_answers)
    ])

    # Compute accuracy
    accuracy = correct_predictions / len(true_answers)
    return accuracy

# Calculate accuracy
accuracy = compute_accuracy(predictions)
print(f"Accuracy on the validation set: {accuracy * 100:.2f}%")

Accuracy on the validation set: 61.31%
